<a href="https://colab.research.google.com/github/ShanmukhiKairuppala/Diabetic_Retinopathy_Detection/blob/main/Mini_Project_DR_Flask_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask flask-ngrok scikit-image opencv-python-headless pyngrok


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install scikit-image tensorflow flask-ngrok
!pip install flask flask-ngrok scikit-image opencv-python-headless pyngrok

import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, img_as_ubyte
import cv2
from flask import Flask, request, jsonify, send_file
import tensorflow as tf
import subprocess
from pyngrok import ngrok
import base64

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static/uploads/'

# Load your model
model = tf.keras.models.load_model('/content/drive/MyDrive/FineTunedModel')

def preprocess_image(image, size=(256, 256)):
    if image.ndim == 2 or image.max() == image.min():
        print("Skipping image: Image is either grayscale or completely black.")
        return None

    if image.ndim == 3 and image.shape[2] == 3:
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl = clahe.apply(l)
        limg = cv2.merge((cl, a, b))
        image = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    elif image.ndim == 2:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        image = clahe.apply(image)

    resized_image = cv2.resize(image, size)
    return img_as_ubyte(resized_image)

def augment_image(image, base_filename, output_dir, retinopathy=True):
    transformations = [image]
    if retinopathy:
        for angle in [90, 120, 180, 270]:
            rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
            transformations.append(rotated_image)
    mirrored_image = cv2.flip(image, 1)
    transformations.append(mirrored_image)
    augmented_filenames = []
    for idx, transformed_image in enumerate(transformations):
        transformed_filename = f"{base_filename}_{idx}.png"
        transformed_path = os.path.join(output_dir, transformed_filename)
        io.imsave(transformed_path, transformed_image)
        augmented_filenames.append(transformed_filename)
    return augmented_filenames

def preprocess_and_augment_single_image(image_path, output_dir, base_filename):
    image = io.imread(image_path)
    preprocessed_image = preprocess_image(image)
    if preprocessed_image is not None:
        augmented_filenames = augment_image(preprocessed_image, base_filename, output_dir, True)
        augmented_images = [io.imread(os.path.join(output_dir, filename)) for filename in augmented_filenames]
        return preprocessed_image, augmented_images
    return None, []

def predict_and_display(model, image_path):
    temp_dir = '/content/temp'
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    preprocessed_image, augmented_images = preprocess_and_augment_single_image(image_path, temp_dir, 'new_image')

    if preprocessed_image is not None:
        image_array = np.expand_dims(preprocessed_image / 255.0, axis=0)
        prediction = model.predict(image_array)
        predicted_class = np.argmax(prediction)
        dr_levels = ["No DR", "Mild DR", "Moderate DR", "Severe DR", "Proliferative DR"]

        num_images = 1 + len(augmented_images)
        plt.figure(figsize=(15, 5))

        plt.subplot(1, num_images, 1)
        plt.imshow(preprocessed_image)
        plt.title('Input Image')

        for i, augmented_image in enumerate(augmented_images):
            plt.subplot(1, num_images, i + 2)
            plt.imshow(augmented_image)
            plt.title(f'Augmented {i + 1}')

        plt.suptitle(f'Predicted DR Level: {predicted_class} ({dr_levels[predicted_class]})')
        result_path = os.path.join(temp_dir, 'result.png')
        plt.savefig(result_path)
        plt.close()

        return result_path, preprocessed_image, augmented_images, predicted_class
    else:
        return None, None, [], None

def encode_image(image):
    _, buffer = cv2.imencode('.png', image)
    encoded_image = base64.b64encode(buffer).decode('ascii')
    return encoded_image

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['file']
    temp_file_path = '/content/temp_uploaded_image.png'
    file.save(temp_file_path)

    result_path, preprocessed_image, augmented_images, predicted_class = predict_and_display(model, temp_file_path)

    if result_path is not None:
        encoded_preprocessed_image = encode_image(preprocessed_image)
        encoded_augmented_images = [encode_image(img) for img in augmented_images]

        return jsonify({
            'prediction': int(predicted_class),
            'preprocessed_image': encoded_preprocessed_image,
            'augmented_images': encoded_augmented_images,
            'result_image_path': result_path
        })
    else:
        return jsonify({'error': 'Image processing failed.'})

if __name__ == '__main__':
    # Start ngrok tunnel
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")

    # Run Flask app
    app.run()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Public URL: NgrokTunnel: "https://1fbe-34-73-107-180.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 [==============================] - 2s 2s/step


INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 14:51:35] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 194ms/step


...

INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 15:44:47] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 197ms/step


INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 15:50:00] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 228ms/step


INFO:werkzeug:127.0.0.1 - - [05/Jun/2024 15:58:19] "POST /predict HTTP/1.1" 200 -
